In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk 
import os
import csv, bz2
import tqdm
from tqdm import tqdm_notebook
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import pickle
import json
import re

In [2]:
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [3]:
import multiprocessing
import gensim.models.word2vec as w2v

### Get 4 part with sentences

In [2]:
sentences = []

In [3]:
for i in tqdm_notebook(range(1,5)):
    with open('filt_sent_%s_vv2.txt' % i) as json_data:
        sentences.append(json.load(json_data))

### Unite in all

In [8]:
all_sentences = []

In [9]:
for part in tqdm_notebook(sentences):
    for sent in part:
        all_sentences.append(sent)

In [10]:
len(all_sentences)

7797905

### Train W2V

In [13]:
num_features = 300
min_word_count = 3
num_workers = multiprocessing.cpu_count()
context_size = 7
downsampling = 1e-3
seed = 1

In [14]:
lenta2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size, 
    sample=downsampling
)

In [16]:
lenta2vec.build_vocab(all_sentences)

In [17]:
print("Word2Vec vocabulary length:", len(lenta2vec.wv.vocab))

Word2Vec vocabulary length: 536700


In [18]:
%%time
lenta2vec.train(all_sentences, 
                 total_examples=lenta2vec.corpus_count,
                 epochs=5)

CPU times: user 7h 29min 34s, sys: 2min 7s, total: 7h 31min 42s
Wall time: 57min 45s


(469545158, 487708280)

#### Save .w2v 

In [19]:
lenta2vec.save('lenta2vec.w2v')

#### Use .w2v

In [4]:
lenta2vec = w2v.Word2Vec.load('lenta2vec.w2v')

In [38]:
lenta2vec.most_similar("инвестиции")

/home/ubuntu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
/home/ubuntu/anaconda3/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('вложения', 0.7722173929214478),
 ('капиталовложения', 0.7324094176292419),
 ('инвестиций', 0.7269445061683655),
 ('инвестировать', 0.7008956074714661),
 ('портфельные', 0.6723456382751465),
 ('вложений', 0.6600357890129089),
 ('инвестирует', 0.6562414765357971),
 ('окупит', 0.6474831104278564),
 ('вкладывать', 0.6447849273681641),
 ('инвестпроекты', 0.6433960199356079)]